In [48]:
import polars as pl
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile
from pathlib import Path


sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

BRONZE_DIR = Path("C:\\Users\\rajpu\\OneDrive\\Documents\\Projects_git\\blood-supply-risk-monitor\\data\\bronze\\ncdb")



In [49]:
files = list(BRONZE_DIR.glob("*.csv"))
print(f"Loading all CSV files from: {BRONZE_DIR}")
print(f"Found {len(files)} files to merge.")
file_strings = [str(f) for f in files]

Loading all CSV files from: C:\Users\rajpu\OneDrive\Documents\Projects_git\blood-supply-risk-monitor\data\bronze\ncdb
Found 23 files to merge.


In [51]:
sample_df = pl.read_csv(
    files[0],
    encoding='latin1',
    n_rows=0,  # Only read headers, no data
    ignore_errors=True
)
expected_columns = sample_df.columns
print(f"Expected columns: {expected_columns}")

Expected columns: ['C_YEAR', 'C_MNTH', 'C_WDAY', 'C_HOUR', 'C_SEV', 'C_VEHS', 'C_CONF', 'C_RCFG', 'C_WTHR', 'C_RSUR', 'C_RALN', 'C_TRAF', 'V_ID', 'V_TYPE', 'V_YEAR', 'P_ID', 'P_SEX', 'P_AGE', 'P_PSN', 'P_ISEV', 'P_SAFE', 'P_USER', 'C_CASE']


In [ ]:
# Check all files for column consistency
all_columns = {}
for file in files:
    df_header = pl.read_csv(
        file,
        encoding='latin1',
        n_rows=0,
        ignore_errors=True
    )
    all_columns[file.name] = df_header.columns
    print(f"{file.name}: {len(df_header.columns)} columns")

# Determine expected columns
column_sets = [set(cols) for cols in all_columns.values()]
if all(cols == column_sets[0] for cols in column_sets):
    expected_columns = list(column_sets[0])
    print(" All files have identical columns")
else:
    print(" Warning: Inconsistent columns")
    expected_columns = sorted(list(set.union(*column_sets)))


ncdb_1999.csv: 23 columns
ncdb_2000.csv: 23 columns
ncdb_2001.csv: 23 columns
ncdb_2002.csv: 23 columns
ncdb_2003.csv: 23 columns
ncdb_2004.csv: 23 columns
ncdb_2005.csv: 23 columns
ncdb_2006.csv: 23 columns
ncdb_2007.csv: 23 columns
ncdb_2008.csv: 23 columns
ncdb_2009.csv: 23 columns
ncdb_2010.csv: 23 columns
ncdb_2011.csv: 23 columns
ncdb_2012.csv: 23 columns
ncdb_2013.csv: 23 columns
ncdb_2014.csv: 23 columns
ncdb_2015.csv: 23 columns
ncdb_2016.csv: 23 columns
ncdb_2017.csv: 23 columns
ncdb_2018.csv: 23 columns
ncdb_2019.csv: 23 columns
ncdb_2020.csv: 23 columns
ncdb_2021.csv: 23 columns
⚠ Warning: Inconsistent columns


In [54]:
expected_columns = sorted(list(set.union(*column_sets)))


In [59]:
columns_to_keep = [
    # Collision features
    'C_YEAR', 'C_MNTH', 'C_WDAY', 'C_HOUR',
    'C_SEV',  # Target variable for severity prediction
    'C_WTHR', 'C_RSUR', 'C_CONF', 'C_TRAF', 
    'C_RCFG', 'C_VEHS', 'C_RALN',
    
    # Person features
    'P_SEX', 'P_AGE', 'P_ISEV', 'P_PSN', 
    'P_SAFE', 'P_USER',
    
    # Vehicle features
    'V_TYPE', 'V_YEAR'
]

In [ ]:
# Filter during merge
dfs = []
for file in files:
    df = pl.read_csv(
        file,
        encoding='latin1',
        null_values=['', 'NA', 'nan', 'U', 'X', 'Q', 'UU', 'XX', 'QQ', 'NN', 'N'],
        ignore_errors=True,
        has_header=True
    )
    
    # Keep only relevant columns that exist in this file
    available_cols = [col for col in columns_to_keep if col in df.columns]
    df = df.select(available_cols)
    dfs.append(df)

merged_df = pl.concat(dfs, how="diagonal") 

In [ ]:
for f in file_strings:
    try:
        # Read file with liberal settings
        temp_df = pl.read_csv(
            f, 
            encoding='latin1',
            null_values=['', 'NA', 'nan', 'U', 'X', 'Q', 'UU', 'XX', 'QQ', 'NN', 'N'],
            ignore_errors=True,
            has_header=True
        )
        
        # FIX: Rename columns if they match the expected count (fixes "2001" header issue)
        if len(temp_df.columns) == len(expected_columns) and temp_df.columns != expected_columns:
            temp_df.columns = expected_columns
            
        # FIX: Explicitly select the 13 columns we need
        # If a file is missing C_CASE, this will fail here (caught by except) 
        # instead of breaking the concat later.
        temp_df = temp_df.select([
            pl.col("C_CASE").cast(pl.Int64, strict=False), 
            pl.col("C_YEAR").cast(pl.Int64, strict=False),
            pl.col("C_MNTH").cast(pl.Int64, strict=False),
            pl.col("C_WDAY").cast(pl.Int64, strict=False),
            pl.col("C_HOUR").cast(pl.Int64, strict=False),
            pl.col("C_SEV").cast(pl.Int64, strict=False),
            pl.col("V_TYPE").cast(pl.String),
            pl.col("V_YEAR").cast(pl.String),
            pl.col("P_SEX").cast(pl.String),
            pl.col("P_AGE").cast(pl.Int64, strict=False),
            pl.col("P_ISEV").cast(pl.String),
            pl.col("P_SAFE").cast(pl.String),
            pl.col("P_USER").cast(pl.String)
        ])
        
        # Safety Check: Ensure width is 13 before appending
        if temp_df.width == 13:
            df_list.append(temp_df)
        else:
            print(f"Skipping {Path(f).name}: Incorrect column count ({temp_df.width})")
        
    except Exception as e:
        print(f"Skipping {Path(f).name}: {e}")

# 3. Concatenate (Should work now as schema is identical)
print(f"Concatenating {len(df_list)} clean DataFrames...")
df_combined = pl.concat(df_list, how="vertical")

print(f"✅ Success! Total Rows: {df_combined.height:,}")
print(f"Columns: {df_combined.columns}")

Concatenating 12 clean DataFrames...


ShapeError: unable to append to a DataFrame of width 8 with a DataFrame of width 13

In [38]:
df_combined = pl.concat(df_list, how="vertical")


print(f"Columns: {df_combined.columns}")


Columns: ['C_YEAR', 'C_MNTH', 'C_WDAY', 'C_SEV', 'P_SEX', 'P_AGE', 'V_TYPE', 'C_HOUR']


In [43]:
df_combined.describe().show()

statistic,C_YEAR,C_MNTH,C_WDAY,C_SEV,P_SEX,P_AGE,V_TYPE,C_HOUR
str,f64,f64,f64,f64,str,f64,str,f64
"""count""",1.622768e6,1.622704e6,1.622187e6,1.622768e6,"""1547242""",1.518118e6,"""1539807""",1.607444e6
"""null_count""",1.0,65.0,582.0,1.0,"""75527""",104651.0,"""82962""",15325.0
"""mean""",2008.221981,6.662242,4.003905,1.982317,null,36.786179,null,13.776286
"""std""",9.891548,3.336505,1.927682,0.131796,null,18.605637,null,5.128423
"""min""",1999.0,1.0,1.0,1.0,"""F""",1.0,"""1""",0.0


In [40]:
# Show the first 5 rows
print("First 5 rows:")
print(df_combined.head())

# Check for missing values in key columns
print("\nMissing Values per Column:")
null_counts = df_combined.null_count()
print(null_counts.transpose())

# Check data types
print("\nColumn Schema:")
print(df_combined.schema)

First 5 rows:
shape: (5, 8)
┌────────┬────────┬────────┬───────┬───────┬───────┬────────┬────────┐
│ C_YEAR ┆ C_MNTH ┆ C_WDAY ┆ C_SEV ┆ P_SEX ┆ P_AGE ┆ V_TYPE ┆ C_HOUR │
│ ---    ┆ ---    ┆ ---    ┆ ---   ┆ ---   ┆ ---   ┆ ---    ┆ ---    │
│ i64    ┆ i64    ┆ i64    ┆ i64   ┆ str   ┆ i64   ┆ str    ┆ i64    │
╞════════╪════════╪════════╪═══════╪═══════╪═══════╪════════╪════════╡
│ 2001   ┆ 4      ┆ 6      ┆ 2     ┆ F     ┆ 28    ┆ 1      ┆ 16     │
│ 2001   ┆ 4      ┆ 6      ┆ 2     ┆ F     ┆ 26    ┆ 1      ┆ 16     │
│ 2001   ┆ 4      ┆ 6      ┆ 2     ┆ M     ┆ 36    ┆ 1      ┆ 16     │
│ 2001   ┆ 4      ┆ 6      ┆ 2     ┆ F     ┆ 68    ┆ 1      ┆ 21     │
│ 2001   ┆ 4      ┆ 6      ┆ 2     ┆ M     ┆ 58    ┆ 1      ┆ 21     │
└────────┴────────┴────────┴───────┴───────┴───────┴────────┴────────┘

Missing Values per Column:
shape: (8, 1)
┌──────────┐
│ column_0 │
│ ---      │
│ u32      │
╞══════════╡
│ 1        │
│ 65       │
│ 582      │
│ 1        │
│ 75527    │
│ 104651   │
│ 8296

In [44]:
# Data Cleaning Steps
print("Cleaning data...")

# 1. Primary Key Check
# Drop rows where C_CASE (Collision Case ID) is missing
# (Based on your stats, C_YEAR has 0 nulls, but C_CASE might have some)
df_clean = df_combined.filter(pl.col('C_CASE').is_not_null())

# 2. Impute Missing Hours
# Calculate median once and fill nulls
median_hour = df_clean.select(pl.col('C_HOUR').median()).item()
df_clean = df_clean.with_columns(
    pl.col('C_HOUR').fill_null(median_hour)
)

# 3. Clean String Columns with "N" / "NN" / "U" values
# We replace these placeholders with actual Nulls so Polars can handle them correctly
df_clean = df_clean.with_columns([
    # Clean P_ISEV (Medical Treatment Required): 'N' -> null, then cast to Int
    pl.when(pl.col('P_ISEV').is_in(['N', 'U', 'X']))
      .then(None)
      .otherwise(pl.col('P_ISEV'))
      .cast(pl.Int64, strict=False)
      .alias('P_ISEV'),

    # Clean P_SAFE (Safety Device Used): 'NN' -> null, then cast to Int
    pl.when(pl.col('P_SAFE').is_in(['NN', 'QQ', 'UU', 'XX']))
      .then(None)
      .otherwise(pl.col('P_SAFE'))
      .cast(pl.Int64, strict=False)
      .alias('P_SAFE'),
      
    # Clean P_SEX: 'U', 'N', 'X' -> null (Keep as String "M"/"F")
    pl.when(pl.col('P_SEX').is_in(['N', 'U', 'X']))
      .then(None)
      .otherwise(pl.col('P_SEX'))
      .alias('P_SEX'),

    # Clean V_YEAR: Cast to Int (invalid strings like "XXXX" become null automatically)
    pl.col('V_YEAR').cast(pl.Int64, strict=False).alias('V_YEAR')
])

# 4. Filter for Valid Ages
# Remove unreasonable ages (e.g., < 0 or > 120) which are likely data errors
df_clean = df_clean.filter(
    (pl.col('P_AGE') >= 0) & (pl.col('P_AGE') <= 120)
)

print(f"Data Cleaned. Final Row Count: {df_clean.height:,}")

# Verification
print("\nMissing values after cleaning:")
print(df_clean.null_count().transpose())

Cleaning data...


ColumnNotFoundError: unable to find column "C_CASE"; valid columns: ["C_YEAR", "C_MNTH", "C_WDAY", "C_SEV", "P_SEX", "P_AGE", "V_TYPE", "C_HOUR"]

## Data Cleaning Process

This section outlines the data cleaning steps applied to the raw NCDB (National Collision Database) data to prepare it for analysis.

### Cleaning Operations Performed:

1. **Primary Key Validation**: Removed any rows with missing `C_CASE` (collision case number) values, as this serves as the primary identifier for each collision record.

2. **Missing Hour Imputation**: Filled missing `C_HOUR` values with the median hour (14.0) to handle temporal data gaps while preserving the distribution.

3. **Categorical Variable Cleaning**:
   - **P_ISEV (Injury Severity)**: Converted 'N' (Not Applicable) codes to null values
   - **P_SAFE (Safety Device Used)**: Converted 'NN' (Not Noted) codes to null values
   - **P_SEX**: Converted 'N' and 'U' (unknown/not reported) codes to null values

4. **Data Type Standardization**:
   - Converted string columns with numeric codes (`P_ISEV`, `P_SAFE`, `V_YEAR`) to appropriate integer types
   - Used strict casting to automatically handle invalid string values (e.g., 'UUUU', 'NNNN') by converting them to null

### Results:
- **Dataset Size**: 272,301 records retained
- **Missing Values Reduced**: Significantly reduced nulls in `C_HOUR` (from 2,149 to 0) and improved data quality in vehicle year information
- **Data Types**: Properly typed columns for downstream analysis (integers for numeric codes, strings for categorical data)


In [ ]:

viz_df = df.select([
    pl.col("C_YEAR").alias("Year"),
    pl.col("C_MNTH").alias("Month"),
    pl.col("C_WDAY").alias("Weekday"),
    
    # FIX: Cast to String first, then replace "1" and "2" (as strings)
    pl.col("C_SEV")
      .cast(pl.String)
      .replace({"1": "Fatal", "2": "Injury"}, default=None)
      .alias("Severity"),

    # P_SEX is already a String column, so this works fine
    pl.col("P_SEX").replace({"M": "Male", "F": "Female", "U": "Unknown"}).alias("Sex"),
    
    pl.col("P_AGE").cast(pl.Int32).alias("Age"),
    pl.col("V_TYPE").alias("Vehicle_Type")
]).to_pandas()

# Filter out rows with missing Years or Severity
viz_df = viz_df.dropna(subset=['Year', 'Severity'])

print("Data prepared for visualization.")
print(viz_df.head())

In [ ]:
trend_data = viz_df.groupby(['Year', 'Severity']).size().reset_index(name='Count')

# 2. Plot
plt.figure(figsize=(12, 6))
sns.lineplot(
    data=trend_data, 
    x='Year', 
    y='Count', 
    hue='Severity', 
    style='Severity', 
    markers=True, 
    dashes=False,
    palette={"Fatal": "#d62728", "Injury": "#1f77b4"}  # Red for Fatal, Blue for Injury
)

plt.title("Total Persons Involved in Collisions by Severity (1999-2023)")
plt.grid(True, alpha=0.3)
plt.ylabel("Number of Persons")
plt.show()

In [ ]:
# 1. Create a pivot table: Rows=Weekday, Cols=Month, Values=Count
heatmap_data = viz_df.pivot_table(
    index='Weekday', 
    columns='Month', 
    values='Year', 
    aggfunc='count'
)

# 2. Reorder weekdays (1=Monday in NCDB usually, but check your specific dict)
# If 1=Monday, 7=Sunday.
plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, cmap="YlOrRd", annot=False, fmt="d")

plt.title("Collision Frequency: Month vs. Day of Week")
plt.ylabel("Day of Week (1=Mon, 7=Sun)")
plt.xlabel("Month")
plt.show()

In [ ]:
# Filter reasonable ages (0-100) to remove outliers/errors
age_df = viz_df[(viz_df['Age'] >= 0) & (viz_df['Age'] <= 100)]

plt.figure(figsize=(12, 6))
sns.histplot(
    data=age_df, 
    x='Age', 
    hue='Sex', 
    multiple="stack", 
    bins=40,
    palette="muted"
)

plt.title("Age Distribution of Persons in Collisions")
plt.xlabel("Age")
plt.ylabel("Count of Persons")
plt.show()